# 3. Solutions to GTFS Reconstruction

This notebook demonstrates converting optimization solutions to data structures that can be written to file. 

The most useful functionality is converting PT solutions back to valid GTFS feeds, but we also extract DRT solutions and write them to JSON files.

The `SolutionExportManager` class in solution_manager.py handles the conversion and writing of solutions. It wraps around two main classes:
- `transit_opt.gtfs.SolutionConverter`: Converts PT optimization solutions to GTFS format. This includes converting headway matrices back to trips and stop times.
- `transit_opt.drt.DRTSolutionExporter`: Exports DRT optimization solutions to JSON files. Each json file has a fleet size for each time interval

In this notebook, we show functionality for writing both:
- A PT solution to GTFS
- A combined PT + DRT solution to GTFS + JSON

To do this, we have to run two example optimization problems, one for PT only, and one for PT + DRT. We then extract the top solutions from each run and write them to file.

In [1]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import geopandas as gpd


# Add src to path
project_root = Path.cwd().parent
src_path = project_root / "src"
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))


print("=== GTFS RECONSTRUCTION WORKFLOW ===")
print("🔄 Testing solution-to-GTFS conversion pipeline")
print(f"📁 Project root: {project_root}")

=== GTFS RECONSTRUCTION WORKFLOW ===
🔄 Testing solution-to-GTFS conversion pipeline
📁 Project root: /home/hussein/Documents/GitHub/transit_opt


## 3.1 Quick Problem Setup

Create a minimal optimization problem to generate test solutions.

In [2]:
from transit_opt.preprocessing.prepare_gtfs import GTFSDataPreparator
from transit_opt.optimisation.config.config_manager import OptimizationConfigManager
from transit_opt.optimisation.runners.pso_runner import PSORunner

print("=== LOADING GTFS DATA ===")

# Quick GTFS setup
preparator = GTFSDataPreparator(
    gtfs_path='../data/external/study_area_gtfs_bus.zip',
    interval_hours=6,  # 4 periods per day for simplicity
    date=None,
    turnaround_buffer=1.15,
    max_round_trip_minutes=240.0,
    no_service_threshold_minutes=480.0,
    log_level="INFO"
)

2025-10-27 11:36:43,650 - transit_opt.preprocessing.prepare_gtfs - INFO - Initializing GTFSDataPreparator with 6h intervals
2025-10-27 11:36:43,651 - transit_opt.preprocessing.prepare_gtfs - INFO - Loading GTFS feed from ../data/external/study_area_gtfs_bus.zip


=== LOADING GTFS DATA ===


2025-10-27 11:36:46,351 - transit_opt.preprocessing.prepare_gtfs - INFO - Using full GTFS feed (all service periods)
2025-10-27 11:36:47,975 - transit_opt.preprocessing.prepare_gtfs - INFO - GTFS loaded and cached in 4.32 seconds
2025-10-27 11:36:47,976 - transit_opt.preprocessing.prepare_gtfs - INFO - Dataset: 13,974 trips, 703,721 stop times


### Extract optimzation data for PT problem

In [3]:
# Simple headway choices for testing
allowed_headways = [10, 15, 30, 60, 120]
opt_data = preparator.extract_optimization_data(allowed_headways)

print(f"✅ Loaded {opt_data['n_routes']} routes, {opt_data['n_intervals']} intervals")
print(f"📊 Headway choices: {allowed_headways}")
print(f"🎯 Decision variables: {np.prod(opt_data['decision_matrix_shape'])}")

2025-10-27 11:36:47,982 - transit_opt.preprocessing.prepare_gtfs - INFO - Extracting optimization data with 5 allowed headways
2025-10-27 11:36:47,984 - transit_opt.preprocessing.prepare_gtfs - INFO - Extracting route essentials with 6-hour intervals
2025-10-27 11:36:49,235 - transit_opt.preprocessing.prepare_gtfs - WARNING - Route 73302: Round-trip 317.4min exceeds limit (240.0min), filtered out
2025-10-27 11:36:49,271 - transit_opt.preprocessing.prepare_gtfs - WARNING - Route 54721: Round-trip 366.8min exceeds limit (240.0min), filtered out
2025-10-27 11:36:50,865 - transit_opt.preprocessing.prepare_gtfs - WARNING - Route 30922: Round-trip 247.2min exceeds limit (240.0min), filtered out
2025-10-27 11:36:51,569 - transit_opt.preprocessing.prepare_gtfs - WARNING - Route 12490: Round-trip 416.3min exceeds limit (240.0min), filtered out
2025-10-27 11:36:51,641 - transit_opt.preprocessing.prepare_gtfs - WARNING - Route 37599: Round-trip 355.3min exceeds limit (240.0min), filtered out
2025

✅ Loaded 147 routes, 4 intervals
📊 Headway choices: [10, 15, 30, 60, 120]
🎯 Decision variables: 588


### Extract optimization data for PT + DRT problem

In [4]:
print("\n=== EXTRACTING PT+DRT OPTIMIZATION DATA ===")


# DRT configuration
drt_config = {
    'enabled': True,
    'target_crs': 'EPSG:3857',  # Web Mercator for consistency
    'default_drt_speed_kmh': 25.0,  # Default speed for all DRT zones
    'zones': [
        {
            'zone_id': 'drt_ne',
            'service_area_path': '../data/external/drt/drt_ne.shp',
            'allowed_fleet_sizes': [0, 10, 25, 50, 100],  # Fleet options for this zone
            'zone_name': 'Leeds NE DRT',
            'drt_speed_kmh': 20.0  # Zone-specific speed (campus area - slower)
        },
        {
            'zone_id': 'drt_nw',
            'service_area_path': '../data/external/drt/drt_nw.shp',
            'allowed_fleet_sizes': [0, 15, 30, 60, 120],  # Different fleet options
            'zone_name': 'Leeds NW DRT'
            # Will use default_drt_speed_kmh (25.0) since zone-specific not provided
        }
    ]
}


# Extract optimization data with DRT support
opt_data_drt = preparator.extract_optimization_data_with_drt(
    allowed_headways=allowed_headways, 
    drt_config=drt_config
)

print(f"\n✅ PT+DRT OPTIMIZATION DATA EXTRACTED:")
print(f"   📊 PT Routes: {opt_data_drt['n_routes']}")
print(f"   🚁 DRT Zones: {opt_data_drt['n_drt_zones']}")
print(f"   ⏰ Time intervals: {opt_data_drt['n_intervals']} ({opt_data_drt['intervals']['duration_minutes']} min each)")
print(f"   🎯 Total decision variables: {opt_data_drt['total_decision_variables']}")
print(f"      • PT variables: {opt_data_drt['pt_decision_variables']}")
print(f"      • DRT variables: {opt_data_drt['drt_decision_variables']}")
print(f"   🔢 PT headway choices: {opt_data_drt['n_choices']}")
print(f"   🚗 Current peak fleet: {opt_data_drt['constraints']['fleet_analysis']['total_current_fleet_peak']} vehicles")

# Verify DRT zones loaded correctly
print(f"\n🗺️ DRT SPATIAL DATA:")
for zone in opt_data_drt['drt_config']['zones']:
    print(f"   Zone {zone['zone_id']}: {zone['area_km2']:.2f} km², speed {zone['drt_speed_kmh']} km/h")

2025-10-27 11:37:04,944 - transit_opt.preprocessing.prepare_gtfs - INFO - Extracting optimization data with 5 allowed headways
2025-10-27 11:37:04,945 - transit_opt.preprocessing.prepare_gtfs - INFO - Extracting route essentials with 6-hour intervals



=== EXTRACTING PT+DRT OPTIMIZATION DATA ===
🔧 EXTRACTING OPTIMIZATION DATA WITH DRT SUPPORT:


2025-10-27 11:37:07,382 - transit_opt.preprocessing.prepare_gtfs - WARNING - Route 73302: Round-trip 317.4min exceeds limit (240.0min), filtered out
2025-10-27 11:37:07,454 - transit_opt.preprocessing.prepare_gtfs - WARNING - Route 54721: Round-trip 366.8min exceeds limit (240.0min), filtered out
2025-10-27 11:37:09,104 - transit_opt.preprocessing.prepare_gtfs - WARNING - Route 30922: Round-trip 247.2min exceeds limit (240.0min), filtered out
2025-10-27 11:37:09,801 - transit_opt.preprocessing.prepare_gtfs - WARNING - Route 12490: Round-trip 416.3min exceeds limit (240.0min), filtered out
2025-10-27 11:37:09,880 - transit_opt.preprocessing.prepare_gtfs - WARNING - Route 37599: Round-trip 355.3min exceeds limit (240.0min), filtered out
2025-10-27 11:37:11,824 - transit_opt.preprocessing.prepare_gtfs - WARNING - Route 59129: Round-trip 396.7min exceeds limit (240.0min), filtered out
2025-10-27 11:37:15,145 - transit_opt.preprocessing.prepare_gtfs - WARNING - Route 73828: Round-trip 258.8

   ✅ Base PT data extracted: 147 routes, 4 intervals
   🚁 Adding DRT configuration...
   🔍 Validating DRT configuration...
   ✅ DRT configuration valid: 2 zones
      Target CRS: EPSG:3857
   🗺️ Loading DRT spatial layers...
      Target CRS: EPSG:3857
      Loading zone 1: drt_ne
         Path: ../data/external/drt/drt_ne.shp
         Original CRS: EPSG:3857
         🔄 Converting: EPSG:3857 → EPSG:3857
   DRT Zone drt_ne: 474.95 km², speed 20.0 km/h
         ✅ Loaded: 474.95 km² service area
            CRS: EPSG:3857
            Fleet choices: [0, 10, 25, 50, 100]
      Loading zone 2: drt_nw
         Path: ../data/external/drt/drt_nw.shp
         Original CRS: EPSG:3857
         🔄 Converting: EPSG:3857 → EPSG:3857
   DRT Zone drt_ne: 474.95 km², speed 20.0 km/h
   DRT Zone drt_nw: 151.69 km², speed 25.0 km/h
         ✅ Loaded: 151.69 km² service area
            CRS: EPSG:3857
            Fleet choices: [0, 15, 30, 60, 120]
   ✅ All DRT spatial layers loaded successfully
      Total

In [5]:
print("\n=== SPATIAL BOUNDARY SETUP ===")

from transit_opt.optimisation.spatial.boundaries import StudyAreaBoundary

# Load boundary geometry (same as basic notebook)
boundary_gdf = gpd.read_file("../data/external/boundaries/study_area_boundary.geojson")
print(f"📍 Loaded boundary with {len(boundary_gdf)} feature(s)")

# Create study area boundary with buffer
study_boundary = StudyAreaBoundary(
    boundary_gdf=boundary_gdf,
    crs="EPSG:3857",  # Web Mercator for spatial analysis
    buffer_km=2.0     # 2km buffer around boundary
)

print(f"✅ Study area boundary created:")
print(f"   📐 CRS: {study_boundary.target_crs}")
print(f"   📏 Buffer: 2km")


=== SPATIAL BOUNDARY SETUP ===
📍 Loaded boundary with 2607 feature(s)
✅ Validated metric CRS: EPSG:3857
🔄 Converting boundary CRS: EPSG:4326 → EPSG:3857
📏 Applied 2.0km buffer
✅ Study area set: 1 polygon(s) in EPSG:3857
✅ Study area boundary created:
   📐 CRS: EPSG:3857
   📏 Buffer: 2km


## 3.2 Writing to file

We have a `SolutionExportManager` class that handles exporting optimization results to files. It works with both `PT` only solutions and `PT + DRT` solutions. PT solutions are saved a zipped gtfs files and DRT solutions are saved as json files. 

When an optimization run includes both PT and DRT services, the resulting solution typically consists of two parts:
1. solution["pt"]: This is an n_interval X n_routes matrix representing the PT component of the solution.
2. solution["drt"]: This is an n_interval X n_zones matrix representing the DRT component of the solution

We use gtfs.py to convert the pt part to gtfs, and drt.py to convert the drt part to json format. The `SolutionExportManager` checks if a solution contains 'pt' only or 'pt' and 'drt', and calls the appropriate export functions.


### PT only problems

#### Run a PT optimization problem

In [6]:
# Quick optimization to get test solutions
print("=== RUNNING QUICK OPTIMIZATION ===")

# Minimal config for fast testing
config_pso_pt = {
    'problem': {
        'objective': {
            'type': 'HexagonalCoverageObjective',
            'spatial_resolution_km': 2.0,  # Larger zones for speed
            'crs': 'EPSG:3857',
            'boundary_file': study_boundary,
            'boundary_buffer_km': 2.0
        },
        'constraints': [
            {
                'type': 'FleetTotalConstraintHandler',
                'baseline': 'current_peak',
                'tolerance': 0.3,  # Max 20% more than current
                'measure': 'peak'
            },
            {
                'type': 'MinimumFleetConstraintHandler',
                'min_fleet_fraction': 0.8,  # Maintain 80% of current service
                'level': 'system',
                'measure': 'peak', 
                'baseline': 'current_peak'
            }
        ]
    },
    'optimization': {
        'algorithm': {
            'type': 'PSO',
            'pop_size': 25,  # Small for speed
            'inertia_weight': 0.9,
            'inertia_weight_final': 0.4,
            'cognitive_coeff': 2.0,
            'social_coeff': 2.0,
            'use_penalty_method': False
        },
        'termination': {
            'max_generations': 30  # Very short for testing
        },
        'monitoring': {
            'progress_frequency': 5,
            'save_history': True,
            'detailed_logging': False
        }
    }
}

# Run optimization
config_manager = OptimizationConfigManager(config_dict=config_pso_pt)
pso_runner = PSORunner(config_manager)
result = pso_runner.optimize(opt_data, track_best_n=5)

print(f"✅ Optimization complete: {result.best_objective:.4f}")
print(f"📊 Found {len(result.best_feasible_solutions) if hasattr(result, 'best_feasible_solutions') and result.best_feasible_solutions else 0} feasible solutions")
print(f"⏱️  Time: {result.optimization_time:.1f}s")

=== RUNNING QUICK OPTIMIZATION ===
📋 Using provided configuration dictionary
🚀 STARTING PSO OPTIMIZATION
🗺️ Setting up spatial analysis with 2.0km resolution
🗺️  Reprojected 6897 stops to EPSG:3857
🔧 Creating 219 × 387 = 84753 grid cells
   Grid bounds: (-452551, 6584519) to (-15585, 7357992) meters
   Cell size: 2000.0m × 2000.0m
✅ Created 84753 hexagonal zones in EPSG:3857
🚀 Using spatial join for zone mapping...
✅ Mapped 6897 stops to zones
🚀 Pre-computing route-stop mappings...
✅ Cached stops for 187 routes
✅ Spatial system ready: 84753 hexagonal zones
   📋 Creating 2 constraint handler(s)...
      Creating constraint 1: FleetTotalConstraintHandler
         ✓ FleetTotal: 1 constraint(s)
      Creating constraint 2: MinimumFleetConstraintHandler
         ✓ MinimumFleet: 1 constraint(s)
🏗️  CREATING TRANSIT OPTIMIZATION PROBLEM:
   📊 Problem dimensions (PT-only):
      Routes: 147
      Time intervals: 4
      Headway choices: 6
   🔧 Pymoo parameters:
      Decision variables: 588
  

#### Extract top solutions
We extract the top N feasible solutions from the optimization result for exporting.

In [7]:
# Extract top N solutions from optimization results
print("=== PROCESSING TOP N SOLUTIONS ===")

# Get the best solutions (you can adjust N as needed)
N_SOLUTIONS = 5
top_solutions = []

if hasattr(result, 'best_feasible_solutions') and result.best_feasible_solutions:
    # Use feasible solutions if available
    top_solutions = result.best_feasible_solutions[:N_SOLUTIONS]
    print(f"📊 Using top {len(top_solutions)} feasible solutions")
else:
    # Fall back to best solution
    top_solutions = [result.best_solution]
    print(f"📊 Using best solution only (no feasible solutions tracked)")

print(f"✅ Will process {len(top_solutions)} solutions")



=== PROCESSING TOP N SOLUTIONS ===
📊 Using top 5 feasible solutions
✅ Will process 5 solutions


#### Writing to file 

In [8]:
# === PT-ONLY SOLUTION EXPORT ===
from transit_opt.gtfs.solution_manager import SolutionExportManager
from pathlib import Path

print("=" * 70)
print("🧪 PART A: PT-ONLY SOLUTION EXPORT")
print("=" * 70)

# 1. Initialize manager for PT-only problem
print(f"\n🔧 Current optimization data type: {opt_data.get('problem_type', 'PT-only')}")
print(f"   DRT enabled: {opt_data.get('drt_enabled', False)}")

pt_export_manager = SolutionExportManager(opt_data)
print(f"   Manager DRT enabled: {pt_export_manager.drt_enabled}")
print(f"   Manager initialized for: {'PT+DRT' if pt_export_manager.drt_enabled else 'PT-only'} problems")

# 2. Prepare solutions in expected format
print(f"\n📊 Preparing {len(top_solutions)} PT-only solutions for export...")
pt_solutions_for_export = []

for i, sol in enumerate(top_solutions, 1):
    solution_matrix = sol['solution'] if isinstance(sol, dict) and 'solution' in sol else sol
    pt_solutions_for_export.append({
        'solution': solution_matrix,  # numpy array for PT-only
        'objective': sol['objective'] if isinstance(sol, dict) and 'objective' in sol else f'rank_{i}'
    })
    print(f"   Solution {i}: shape {solution_matrix.shape}, objective {sol.get('objective', 'N/A')}")

# 3. Export PT-only solutions to directory
output_dir = "output/pt_only_solutions"
print(f"\n🏗️  Exporting PT-only solutions to: {output_dir}")

pt_export_results = pt_export_manager.export_solution_set(
    solutions=pt_solutions_for_export,
    base_output_dir=output_dir,
    solution_prefix="pt_solution",
    metadata=None  # Use directory structure for organization
)

# 4. Display PT-only results
print(f"\n📁 EXPORTED {len(pt_export_results)} PT-ONLY SOLUTIONS:")
print(f"{'Solution ID':<15} {'GTFS File':<30} {'Objective':<12}")
print("-" * 60)

for result in pt_export_results:
    solution_id = result['solution_id']
    pt_export = result['exports']['pt']
    pt_filename = Path(pt_export['path']).name
    objective = result['metadata'].get('objective_value', 'N/A')
    objective_str = f"{objective:.4f}" if isinstance(objective, (int, float)) else str(objective)
    
    print(f"{solution_id:<15} {pt_filename:<30} {objective_str:<12}")

print(f"\n📂 PT-Only File Structure Created:")
print(f"   {output_dir}/")
for result in pt_export_results:
    pt_filename = Path(result['exports']['pt']['path']).name
    print(f"   ├── {pt_filename}")
    
print("\n✅ PT-only solution export completed!")

🧪 PART A: PT-ONLY SOLUTION EXPORT

🔧 Current optimization data type: discrete_headway_optimization
   DRT enabled: False
   Manager DRT enabled: False
   Manager initialized for: PT-only problems

📊 Preparing 5 PT-only solutions for export...
   Solution 1: shape (147, 4), objective 8.644050341936415
   Solution 2: shape (147, 4), objective 8.652294782517718
   Solution 3: shape (147, 4), objective 8.750457479867716
   Solution 4: shape (147, 4), objective 8.791451715290291
   Solution 5: shape (147, 4), objective 8.801018318319677

🏗️  Exporting PT-only solutions to: output/pt_only_solutions
🔍 Processing route 50627...
   ✅ 00-06h: 34.0min template
   ✅ 06-12h: 43.0min template
   ✅ 12-18h: 43.0min template
   ✅ 18-24h: 35.0min template
🔍 Processing route 50628...
   ✅ 00-06h: 65.0min template
   ✅ 06-12h: 79.0min template
   ✅ 12-18h: 80.0min template
   ✅ 18-24h: 64.0min template
🔍 Processing route 50629...
   ✅ 00-06h: 72.0min template
   ✅ 06-12h: 72.0min template
   ✅ 12-18h: 74.

Found 155 stops with invalid parent_station references: <StringArray>
[    '450G2617',     '450G7920',     '450G2572',     '450G2393',
     '450G7954',     '450G7613',     '450G6820',     '450G8341',
     '450G7637',     '450G7935',     '450G1142',     '450G7922',
     '450G9423',     '450G9566',     '450G8193',     '029G0052',
  '049GBUSCWY1',    '075G71047',    '079G73001', '109GDDCCBS01',
     '180GCSBS',     '180GMABS',     '180GSHIC',  '269GLC30614',
 '280G00000005',   '330GMA0337',     '339GBB08', '340G00001090',
   '370G100007',   '370G100004',   '370G105120',   '370G100009',
   '380G510101',    '430G00050',    '430G01055',    '430G21031',
   '440GCY0359',     '450G5168',     '450G6011',     '450G7949',
     '450G8049',    '450G21825',    '450G22583',     '450G7924',
     '450G7936',     '450G9478',     '450G8061',     '450G9178',
     '450G8028',  '910GHTRWCBS',  '910GHTRBUS5',     '910GPBRO']
Length: 52, dtype: string


⚠️  Found 155 stops with invalid parent_station references
   Missing parent stations: ['450G2617', '450G7920', '450G2572', '450G2393', '450G7954', '450G7613', '450G6820', '450G8341', '450G7637', '450G7935', '450G1142', '450G7922', '450G9423', '450G9566', '450G8193', '029G0052', '049GBUSCWY1', '075G71047', '079G73001', '109GDDCCBS01', '180GCSBS', '180GMABS', '180GSHIC', '269GLC30614', '280G00000005', '330GMA0337', '339GBB08', '340G00001090', '370G100007', '370G100004', '370G105120', '370G100009', '380G510101', '430G00050', '430G01055', '430G21031', '440GCY0359', '450G5168', '450G6011', '450G7949', '450G8049', '450G21825', '450G22583', '450G7924', '450G7936', '450G9478', '450G8061', '450G9178', '450G8028', '910GHTRWCBS', '910GHTRBUS5', '910GPBRO']
✅ Cleared 155 invalid parent_station references
✅ Fixed and copied stops.txt: 6897 stops
✅ Copied routes.txt: 187 routes
✅ Copied agency.txt: 24 agencies
✅ Generated trips.txt: 7307 trips
✅ Generated stop_times.txt: 317871 stop times
📅 Using m

Found 155 stops with invalid parent_station references: <StringArray>
[    '450G2617',     '450G7920',     '450G2572',     '450G2393',
     '450G7954',     '450G7613',     '450G6820',     '450G8341',
     '450G7637',     '450G7935',     '450G1142',     '450G7922',
     '450G9423',     '450G9566',     '450G8193',     '029G0052',
  '049GBUSCWY1',    '075G71047',    '079G73001', '109GDDCCBS01',
     '180GCSBS',     '180GMABS',     '180GSHIC',  '269GLC30614',
 '280G00000005',   '330GMA0337',     '339GBB08', '340G00001090',
   '370G100007',   '370G100004',   '370G105120',   '370G100009',
   '380G510101',    '430G00050',    '430G01055',    '430G21031',
   '440GCY0359',     '450G5168',     '450G6011',     '450G7949',
     '450G8049',    '450G21825',    '450G22583',     '450G7924',
     '450G7936',     '450G9478',     '450G8061',     '450G9178',
     '450G8028',  '910GHTRWCBS',  '910GHTRBUS5',     '910GPBRO']
Length: 52, dtype: string


⚠️  Found 155 stops with invalid parent_station references
   Missing parent stations: ['450G2617', '450G7920', '450G2572', '450G2393', '450G7954', '450G7613', '450G6820', '450G8341', '450G7637', '450G7935', '450G1142', '450G7922', '450G9423', '450G9566', '450G8193', '029G0052', '049GBUSCWY1', '075G71047', '079G73001', '109GDDCCBS01', '180GCSBS', '180GMABS', '180GSHIC', '269GLC30614', '280G00000005', '330GMA0337', '339GBB08', '340G00001090', '370G100007', '370G100004', '370G105120', '370G100009', '380G510101', '430G00050', '430G01055', '430G21031', '440GCY0359', '450G5168', '450G6011', '450G7949', '450G8049', '450G21825', '450G22583', '450G7924', '450G7936', '450G9478', '450G8061', '450G9178', '450G8028', '910GHTRWCBS', '910GHTRBUS5', '910GPBRO']
✅ Cleared 155 invalid parent_station references
✅ Fixed and copied stops.txt: 6897 stops
✅ Copied routes.txt: 187 routes
✅ Copied agency.txt: 24 agencies
✅ Generated trips.txt: 7209 trips
✅ Generated stop_times.txt: 315253 stop times
📅 Using m

...

⚠️  Found 155 stops with invalid parent_station references
   Missing parent stations: ['450G2617', '450G7920', '450G2572', '450G2393', '450G7954', '450G7613', '450G6820', '450G8341', '450G7637', '450G7935', '450G1142', '450G7922', '450G9423', '450G9566', '450G8193', '029G0052', '049GBUSCWY1', '075G71047', '079G73001', '109GDDCCBS01', '180GCSBS', '180GMABS', '180GSHIC', '269GLC30614', '280G00000005', '330GMA0337', '339GBB08', '340G00001090', '370G100007', '370G100004', '370G105120', '370G100009', '380G510101', '430G00050', '430G01055', '430G21031', '440GCY0359', '450G5168', '450G6011', '450G7949', '450G8049', '450G21825', '450G22583', '450G7924', '450G7936', '450G9478', '450G8061', '450G9178', '450G8028', '910GHTRWCBS', '910GHTRBUS5', '910GPBRO']
✅ Cleared 155 invalid parent_station references
✅ Fixed and copied stops.txt: 6897 stops
✅ Copied routes.txt: 187 routes
✅ Copied agency.txt: 24 agencies
✅ Generated trips.txt: 7262 trips
✅ Generated stop_times.txt: 317226 stop times
📅 Using m

Found 155 stops with invalid parent_station references: <StringArray>
[    '450G2617',     '450G7920',     '450G2572',     '450G2393',
     '450G7954',     '450G7613',     '450G6820',     '450G8341',
     '450G7637',     '450G7935',     '450G1142',     '450G7922',
     '450G9423',     '450G9566',     '450G8193',     '029G0052',
  '049GBUSCWY1',    '075G71047',    '079G73001', '109GDDCCBS01',
     '180GCSBS',     '180GMABS',     '180GSHIC',  '269GLC30614',
 '280G00000005',   '330GMA0337',     '339GBB08', '340G00001090',
   '370G100007',   '370G100004',   '370G105120',   '370G100009',
   '380G510101',    '430G00050',    '430G01055',    '430G21031',
   '440GCY0359',     '450G5168',     '450G6011',     '450G7949',
     '450G8049',    '450G21825',    '450G22583',     '450G7924',
     '450G7936',     '450G9478',     '450G8061',     '450G9178',
     '450G8028',  '910GHTRWCBS',  '910GHTRBUS5',     '910GPBRO']
Length: 52, dtype: string


⚠️  Found 155 stops with invalid parent_station references
   Missing parent stations: ['450G2617', '450G7920', '450G2572', '450G2393', '450G7954', '450G7613', '450G6820', '450G8341', '450G7637', '450G7935', '450G1142', '450G7922', '450G9423', '450G9566', '450G8193', '029G0052', '049GBUSCWY1', '075G71047', '079G73001', '109GDDCCBS01', '180GCSBS', '180GMABS', '180GSHIC', '269GLC30614', '280G00000005', '330GMA0337', '339GBB08', '340G00001090', '370G100007', '370G100004', '370G105120', '370G100009', '380G510101', '430G00050', '430G01055', '430G21031', '440GCY0359', '450G5168', '450G6011', '450G7949', '450G8049', '450G21825', '450G22583', '450G7924', '450G7936', '450G9478', '450G8061', '450G9178', '450G8028', '910GHTRWCBS', '910GHTRBUS5', '910GPBRO']
✅ Cleared 155 invalid parent_station references
✅ Fixed and copied stops.txt: 6897 stops
✅ Copied routes.txt: 187 routes
✅ Copied agency.txt: 24 agencies
✅ Generated trips.txt: 7258 trips
✅ Generated stop_times.txt: 316037 stop times
📅 Using m

Found 155 stops with invalid parent_station references: <StringArray>
[    '450G2617',     '450G7920',     '450G2572',     '450G2393',
     '450G7954',     '450G7613',     '450G6820',     '450G8341',
     '450G7637',     '450G7935',     '450G1142',     '450G7922',
     '450G9423',     '450G9566',     '450G8193',     '029G0052',
  '049GBUSCWY1',    '075G71047',    '079G73001', '109GDDCCBS01',
     '180GCSBS',     '180GMABS',     '180GSHIC',  '269GLC30614',
 '280G00000005',   '330GMA0337',     '339GBB08', '340G00001090',
   '370G100007',   '370G100004',   '370G105120',   '370G100009',
   '380G510101',    '430G00050',    '430G01055',    '430G21031',
   '440GCY0359',     '450G5168',     '450G6011',     '450G7949',
     '450G8049',    '450G21825',    '450G22583',     '450G7924',
     '450G7936',     '450G9478',     '450G8061',     '450G9178',
     '450G8028',  '910GHTRWCBS',  '910GHTRBUS5',     '910GPBRO']
Length: 52, dtype: string


⚠️  Found 155 stops with invalid parent_station references
   Missing parent stations: ['450G2617', '450G7920', '450G2572', '450G2393', '450G7954', '450G7613', '450G6820', '450G8341', '450G7637', '450G7935', '450G1142', '450G7922', '450G9423', '450G9566', '450G8193', '029G0052', '049GBUSCWY1', '075G71047', '079G73001', '109GDDCCBS01', '180GCSBS', '180GMABS', '180GSHIC', '269GLC30614', '280G00000005', '330GMA0337', '339GBB08', '340G00001090', '370G100007', '370G100004', '370G105120', '370G100009', '380G510101', '430G00050', '430G01055', '430G21031', '440GCY0359', '450G5168', '450G6011', '450G7949', '450G8049', '450G21825', '450G22583', '450G7924', '450G7936', '450G9478', '450G8061', '450G9178', '450G8028', '910GHTRWCBS', '910GHTRBUS5', '910GPBRO']
✅ Cleared 155 invalid parent_station references
✅ Fixed and copied stops.txt: 6897 stops
✅ Copied routes.txt: 187 routes
✅ Copied agency.txt: 24 agencies
✅ Generated trips.txt: 7305 trips
✅ Generated stop_times.txt: 317796 stop times
📅 Using m

### PT + DRT problems

#### Run a PT + DRT optimization problem 

In [9]:
print("\n=== RUNNING PT+DRT OPTIMIZATION ===")

# Configuration for PT+DRT optimization (similar to basic notebook)
config_pso_drt = {
    'problem': {
        'objective': {
            'type': 'HexagonalCoverageObjective',
            'spatial_resolution_km': 2.0,
            'crs': 'EPSG:3857',
            'boundary': study_boundary,
            'boundary_buffer_km': 2.0,
            'time_aggregation': 'average'
        },
        'constraints': [
            {
                'type': 'FleetTotalConstraintHandler',
                'baseline': 'current_peak',
                'tolerance': 0.25,  # 35% increase allowed for PT
                'measure': 'peak'
            },
            {
                'type': 'MinimumFleetConstraintHandler',
                'min_fleet_fraction': 0.85,  # Maintain 85% of current PT service
                'level': 'system',
                'measure': 'peak', 
                'baseline': 'current_peak'
            }
        ]
    },
    'optimization': {
        'algorithm': {
            'type': 'PSO',
            'pop_size': 35,         # Smaller population for faster testing
            'inertia_weight': 0.9,
            'inertia_weight_final': 0.4,
            'cognitive_coeff': 2.0,
            'social_coeff': 2.0,
            'use_penalty_method': False
        },
        'termination': {'max_generations': 25},  # Fewer generations for testing
        'monitoring': {'progress_frequency': 5, 'save_history': False}
    }
}

from transit_opt.optimisation.config.config_manager import OptimizationConfigManager
from transit_opt.optimisation.runners.pso_runner import PSORunner

print(f"🚀 OPTIMIZATION CONFIGURATION:")
print(f"   Algorithm: PSO with {config_pso_drt['optimization']['algorithm']['pop_size']} particles")
print(f"   Generations: {config_pso_drt['optimization']['termination']['max_generations']}")
print(f"   Objective: Spatial equity (minimize variance)")
print(f"   DRT zones: {opt_data_drt['n_drt_zones']} with fleet optimization")

config_manager_drt = OptimizationConfigManager(config_dict=config_pso_drt)
pso_runner_drt = PSORunner(config_manager_drt)

# Run optimization
result_drt = pso_runner_drt.optimize(opt_data_drt, track_best_n=3)

print(f"✅ PT+DRT optimization complete: {result_drt.best_objective:.4f}")
print(f"📊 Time: {result_drt.optimization_time:.1f}s")
    


=== RUNNING PT+DRT OPTIMIZATION ===
🚀 OPTIMIZATION CONFIGURATION:
   Algorithm: PSO with 35 particles
   Generations: 25
   Objective: Spatial equity (minimize variance)
   DRT zones: 2 with fleet optimization
📋 Using provided configuration dictionary
🚀 STARTING PSO OPTIMIZATION
🗺️ Setting up spatial analysis with 2.0km resolution
🗺️  Reprojected 6897 stops to EPSG:3857
🎯 Applying boundary filter to 6897 stops...
🔍 Filtered 6897 → 4405 points
✅ Filtered to 4405 stops within boundary
🔧 Creating 27 × 26 = 702 grid cells
   Grid bounds: (-195346, 7111759) to (-142657, 7161976) meters
   Cell size: 2000.0m × 2000.0m
✅ Created 702 hexagonal zones in EPSG:3857
🎯 Applying boundary filter to 702 grid cells...
🔍 Filtered 702 → 552 grid cells
✅ Filtered to 552 grid cells within boundary
🚀 Using spatial join for zone mapping...
✅ Mapped 4405 stops to zones
🗺️ Computing DRT spatial intersections for 2 zones...
   Hexagonal grid size: 552 zones
   Zone drt_ne: affects 149 hexagonal zones
   Zone d

#### Extract top solutions

In [10]:
# Extract top solutions
drt_top_solutions = result_drt.best_feasible_solutions[:3]
print(f"📊 Using top {len(drt_top_solutions)} feasible PT+DRT solutions")

📊 Using top 3 feasible PT+DRT solutions


#### Writing to file

In [11]:
# === PT+DRT SOLUTION EXPORT ===
print("\n🏗️  Exporting PT+DRT combined solutions...")

# 1. Initialize manager for PT+DRT problem
drt_export_manager = SolutionExportManager(opt_data_drt)
print(f"   Manager type: {'PT+DRT' if drt_export_manager.drt_enabled else 'PT-only'}")
print(f"   DRT exporter available: {drt_export_manager.drt_exporter is not None}")

# 2. Prepare combined solutions for export
drt_solutions_for_export = []
for i, sol in enumerate(drt_top_solutions, 1):
    drt_solutions_for_export.append({
        'solution': sol['solution'],  # dict with 'pt' and 'drt' keys
        'objective': sol['objective']
    })
    
    print(f"   Combined solution {i}: objective {sol['objective']:.4f}")

# 3. Export combined solutions
drt_output_dir = "output/combined_pt_drt_solutions"
print(f"\n📁 Exporting to: {drt_output_dir}")

try:
    drt_export_results = drt_export_manager.export_solution_set(
        solutions=drt_solutions_for_export,
        base_output_dir=drt_output_dir,
        solution_prefix="combined_solution",
        metadata=None  # Directory-based organization
    )
    
    # 4. Display combined results
    print(f"\n📁 EXPORTED {len(drt_export_results)} COMBINED PT+DRT SOLUTIONS:")
    print(f"{'Solution ID':<18} {'PT (GTFS)':<25} {'DRT (JSON)':<25} {'Objective':<12}")
    print("-" * 85)
    
    for result in drt_export_results:
        solution_id = result['solution_id']
        pt_export = result['exports']['pt']
        drt_export = result['exports']['drt']
        
        pt_filename = Path(pt_export['path']).name
        drt_filename = Path(drt_export['path']).name
        objective = result['metadata'].get('objective_value', 'N/A')
        objective_str = f"{objective:.4f}" if isinstance(objective, (int, float)) else str(objective)
        
        print(f"{solution_id:<18} {pt_filename:<25} {drt_filename:<25} {objective_str:<12}")
    
    print(f"\n📂 Combined PT+DRT File Structure:")
    print(f"   {drt_output_dir}/")
    for result in drt_export_results:
        pt_filename = Path(result['exports']['pt']['path']).name
        drt_filename = Path(result['exports']['drt']['path']).name
        print(f"   ├── {pt_filename}")
        print(f"   ├── {drt_filename}")
    
    print("\n✅ Combined PT+DRT solution export completed!")
    
    # 5. Show cross-reference example
    print(f"\n🔗 Cross-Reference Example:")
    example_result = drt_export_results[0]
    pt_file = Path(example_result['exports']['pt']['path']).name
    drt_pt_ref = example_result['exports']['drt']['pt_reference']
    print(f"   PT GTFS file: {pt_file}")
    print(f"   DRT PT reference: {drt_pt_ref}")
    print(f"   Cross-reference: {'✅ MATCH' if pt_file == drt_pt_ref else '❌ MISMATCH'}")

except Exception as e:
    print(f"❌ PT+DRT export failed: {e}")
    print("This might occur if DRT configuration is incomplete.")


🏗️  Exporting PT+DRT combined solutions...
   Manager type: PT+DRT
   DRT exporter available: True
   Combined solution 1: objective 1212.8383
   Combined solution 2: objective 1214.1817
   Combined solution 3: objective 1215.8265

📁 Exporting to: output/combined_pt_drt_solutions
🔍 Processing route 50627...
   ✅ 00-06h: 34.0min template
   ✅ 06-12h: 43.0min template
   ✅ 12-18h: 43.0min template
   ✅ 18-24h: 35.0min template
🔍 Processing route 50628...
   ✅ 00-06h: 65.0min template
   ✅ 06-12h: 79.0min template
   ✅ 12-18h: 80.0min template
   ✅ 18-24h: 64.0min template
🔍 Processing route 50629...
   ✅ 00-06h: 72.0min template
   ✅ 06-12h: 72.0min template
   ✅ 12-18h: 74.0min template
   ✅ 18-24h: 63.0min template
🔍 Processing route 58940...
   ⚠️  00-06h: Using fallback template (30.0min)
   ✅ 06-12h: 20.0min template
   ✅ 12-18h: 30.0min template
   ⚠️  18-24h: Using fallback template (30.0min)
🔍 Processing route 22577...
   ✅ 00-06h: 74.0min template
   ✅ 06-12h: 84.0min template


Found 155 stops with invalid parent_station references: <StringArray>
[    '450G2617',     '450G7920',     '450G2572',     '450G2393',
     '450G7954',     '450G7613',     '450G6820',     '450G8341',
     '450G7637',     '450G7935',     '450G1142',     '450G7922',
     '450G9423',     '450G9566',     '450G8193',     '029G0052',
  '049GBUSCWY1',    '075G71047',    '079G73001', '109GDDCCBS01',
     '180GCSBS',     '180GMABS',     '180GSHIC',  '269GLC30614',
 '280G00000005',   '330GMA0337',     '339GBB08', '340G00001090',
   '370G100007',   '370G100004',   '370G105120',   '370G100009',
   '380G510101',    '430G00050',    '430G01055',    '430G21031',
   '440GCY0359',     '450G5168',     '450G6011',     '450G7949',
     '450G8049',    '450G21825',    '450G22583',     '450G7924',
     '450G7936',     '450G9478',     '450G8061',     '450G9178',
     '450G8028',  '910GHTRWCBS',  '910GHTRBUS5',     '910GPBRO']
Length: 52, dtype: string


⚠️  Found 155 stops with invalid parent_station references
   Missing parent stations: ['450G2617', '450G7920', '450G2572', '450G2393', '450G7954', '450G7613', '450G6820', '450G8341', '450G7637', '450G7935', '450G1142', '450G7922', '450G9423', '450G9566', '450G8193', '029G0052', '049GBUSCWY1', '075G71047', '079G73001', '109GDDCCBS01', '180GCSBS', '180GMABS', '180GSHIC', '269GLC30614', '280G00000005', '330GMA0337', '339GBB08', '340G00001090', '370G100007', '370G100004', '370G105120', '370G100009', '380G510101', '430G00050', '430G01055', '430G21031', '440GCY0359', '450G5168', '450G6011', '450G7949', '450G8049', '450G21825', '450G22583', '450G7924', '450G7936', '450G9478', '450G8061', '450G9178', '450G8028', '910GHTRWCBS', '910GHTRBUS5', '910GPBRO']
✅ Cleared 155 invalid parent_station references
✅ Fixed and copied stops.txt: 6897 stops
✅ Copied routes.txt: 187 routes
✅ Copied agency.txt: 24 agencies
✅ Generated trips.txt: 8107 trips
✅ Generated stop_times.txt: 357979 stop times
📅 Using m

Found 155 stops with invalid parent_station references: <StringArray>
[    '450G2617',     '450G7920',     '450G2572',     '450G2393',
     '450G7954',     '450G7613',     '450G6820',     '450G8341',
     '450G7637',     '450G7935',     '450G1142',     '450G7922',
     '450G9423',     '450G9566',     '450G8193',     '029G0052',
  '049GBUSCWY1',    '075G71047',    '079G73001', '109GDDCCBS01',
     '180GCSBS',     '180GMABS',     '180GSHIC',  '269GLC30614',
 '280G00000005',   '330GMA0337',     '339GBB08', '340G00001090',
   '370G100007',   '370G100004',   '370G105120',   '370G100009',
   '380G510101',    '430G00050',    '430G01055',    '430G21031',
   '440GCY0359',     '450G5168',     '450G6011',     '450G7949',
     '450G8049',    '450G21825',    '450G22583',     '450G7924',
     '450G7936',     '450G9478',     '450G8061',     '450G9178',
     '450G8028',  '910GHTRWCBS',  '910GHTRBUS5',     '910GPBRO']
Length: 52, dtype: string


⚠️  Found 155 stops with invalid parent_station references
   Missing parent stations: ['450G2617', '450G7920', '450G2572', '450G2393', '450G7954', '450G7613', '450G6820', '450G8341', '450G7637', '450G7935', '450G1142', '450G7922', '450G9423', '450G9566', '450G8193', '029G0052', '049GBUSCWY1', '075G71047', '079G73001', '109GDDCCBS01', '180GCSBS', '180GMABS', '180GSHIC', '269GLC30614', '280G00000005', '330GMA0337', '339GBB08', '340G00001090', '370G100007', '370G100004', '370G105120', '370G100009', '380G510101', '430G00050', '430G01055', '430G21031', '440GCY0359', '450G5168', '450G6011', '450G7949', '450G8049', '450G21825', '450G22583', '450G7924', '450G7936', '450G9478', '450G8061', '450G9178', '450G8028', '910GHTRWCBS', '910GHTRBUS5', '910GPBRO']
✅ Cleared 155 invalid parent_station references
✅ Fixed and copied stops.txt: 6897 stops
✅ Copied routes.txt: 187 routes
✅ Copied agency.txt: 24 agencies
✅ Generated trips.txt: 8142 trips
✅ Generated stop_times.txt: 358371 stop times
📅 Using m

Found 155 stops with invalid parent_station references: <StringArray>
[    '450G2617',     '450G7920',     '450G2572',     '450G2393',
     '450G7954',     '450G7613',     '450G6820',     '450G8341',
     '450G7637',     '450G7935',     '450G1142',     '450G7922',
     '450G9423',     '450G9566',     '450G8193',     '029G0052',
  '049GBUSCWY1',    '075G71047',    '079G73001', '109GDDCCBS01',
     '180GCSBS',     '180GMABS',     '180GSHIC',  '269GLC30614',
 '280G00000005',   '330GMA0337',     '339GBB08', '340G00001090',
   '370G100007',   '370G100004',   '370G105120',   '370G100009',
   '380G510101',    '430G00050',    '430G01055',    '430G21031',
   '440GCY0359',     '450G5168',     '450G6011',     '450G7949',
     '450G8049',    '450G21825',    '450G22583',     '450G7924',
     '450G7936',     '450G9478',     '450G8061',     '450G9178',
     '450G8028',  '910GHTRWCBS',  '910GHTRBUS5',     '910GPBRO']
Length: 52, dtype: string


⚠️  Found 155 stops with invalid parent_station references
   Missing parent stations: ['450G2617', '450G7920', '450G2572', '450G2393', '450G7954', '450G7613', '450G6820', '450G8341', '450G7637', '450G7935', '450G1142', '450G7922', '450G9423', '450G9566', '450G8193', '029G0052', '049GBUSCWY1', '075G71047', '079G73001', '109GDDCCBS01', '180GCSBS', '180GMABS', '180GSHIC', '269GLC30614', '280G00000005', '330GMA0337', '339GBB08', '340G00001090', '370G100007', '370G100004', '370G105120', '370G100009', '380G510101', '430G00050', '430G01055', '430G21031', '440GCY0359', '450G5168', '450G6011', '450G7949', '450G8049', '450G21825', '450G22583', '450G7924', '450G7936', '450G9478', '450G8061', '450G9178', '450G8028', '910GHTRWCBS', '910GHTRBUS5', '910GPBRO']
✅ Cleared 155 invalid parent_station references
✅ Fixed and copied stops.txt: 6897 stops
✅ Copied routes.txt: 187 routes
✅ Copied agency.txt: 24 agencies
✅ Generated trips.txt: 8225 trips
✅ Generated stop_times.txt: 360963 stop times
📅 Using m